http://localhost:8889/notebooks/git/google-quest-challenge/working/QstBertBase0113_1.ipynb

# start

In [4]:
trntmstmp=1584412344
valtmstmp=1585069785
import datetime
print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (trntmstmp, valtmstmp)])

grand_total=1.5e8
MIN_TM_TRN=1580947200
MIN_TM_TST=1581552000
print([datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (MIN_TM_TRN, MIN_TM_TST)])


CHNKSZ=1e6
POST_RATE_WANTED=0.1

['2020-03-17 02:32:24', '2020-03-24 17:09:45']
['2020-02-06 00:00:00', '2020-02-13 00:00:00']


# setup

In [442]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

SEED=101
HOME='/data/git/recsys20'
p_in=f'{HOME}/input'

import torch
from transformers import *
import torch
device=torch.device('cpu')
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader


pretrained_weights='bert-base-multilingual-cased'
bertmodel = BertModel.from_pretrained(pretrained_weights, output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained(pretrained_weights, do_lower_case=False)


In [443]:
cols=[
'toks',
'hshtgs',
'twtid',
'media',
'links',
'domns',
'twttyp',
'lang',
'tm',

'u1id',
'u1_fllwer_cnt',
'u1_fllwng_cnt',
'u1_vrfed',
'u1_create_tm',

'u2id',
'u2_fllwer_cnt',
'u2_fllwng_cnt',
'u2_vrfed',
'u2_create_tm',

'u1_fllw_u2',
'reply_tm',
'retwt_tm',
'retwt_cmmnt_tm',
'like_tm',
]
cols_cat = ['twttyp','lang']
cols_val = cols[:-4]
cols_tgt_tmstmp=[
    'retwt_tm',
    'reply_tm',
    'like_tm',
    'retwt_cmmnt_tm',
]
cols_tgt=[o.split('_tm')[0] for o in cols_tgt_tmstmp]
tgts             = ['Retweet','Reply','Like','RTwCmnt',]
assert cols_tgt == ['retwt',  'reply','like','retwt_cmmnt',]
ntgts=len(tgts)


tgt2col=dict(zip(tgts,cols_tgt))
tgt2col

{'Retweet': 'retwt',
 'Reply': 'reply',
 'Like': 'like',
 'RTwCmnt': 'retwt_cmmnt'}

# prep

In [444]:
%%time
dftrn=pd.read_csv(
    f'{p_in}/trn_{trntmstmp}.tsv',
    sep='\x01', header=None, 
    encoding='utf-8', 
    names=cols, 
    nrows=1e4
)

CPU times: user 78.8 ms, sys: 0 ns, total: 78.8 ms
Wall time: 78.2 ms


In [445]:
# tokenizer.pad_token, tokenizer.pad_token_id
# ('[PAD]', 0)

# tokenizer.sep_token, tokenizer.sep_token_id
# ('[SEP]', 102)
maxlen=512
def mkids(x):
    tokids=list(map(int, x.split('\t')))
    l=len(tokids) 
    if l<=maxlen: 
        return tokids + [0]*(maxlen-len(tokids))
    else: 
        return tokids[:maxlen-1]+[102]

In [446]:
def mk_tensors(df, istrn=True):
    tokids=dftrn.toks.apply(lambda x: mkids(x))
    Xarr=np.array(list(tokids))
    X=torch.tensor(Xarr,dtype=torch.long)
    masks=torch.tensor(Xarr!=0, dtype=torch.long)
    segments=torch.zeros(X.shape, dtype=torch.long)

    if not istrn:
        return X,masks,segments,ys
    ys=dftrn[cols_tgt_tmstmp].notna().values
    ys=torch.tensor(ys,dtype=torch.float)
    return X,masks,segments,ys
    

In [447]:
X,masks,segments,ys = mk_tensors(dftrn)

In [448]:
X,masks,segments,ys = mk_tensors(dftrn)

In [449]:
ind=np.random.choice(len(X))
print(list(zip(X[ind].numpy(), masks[ind].numpy())))
print(" ".join(tokenizer.convert_ids_to_tokens(X[ind])))
print(masks[ind])

[(101, 1), (56898, 1), (137, 1), (12786, 1), (11779, 1), (27043, 1), (10133, 1), (53398, 1), (131, 1), (75877, 1), (15362, 1), (19626, 1), (10464, 1), (12469, 1), (11661, 1), (20826, 1), (119, 1), (10117, 1), (25157, 1), (10108, 1), (10840, 1), (66807, 1), (10822, 1), (10135, 1), (39422, 1), (100, 1), (187, 1), (13295, 1), (119, 1), (14962, 1), (13172, 1), (75339, 1), (82321, 1), (10957, 1), (10111, 1), (10105, 1), (11661, 1), (119, 1), (65272, 1), (47116, 1), (100, 1), (102, 1), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0,

In [450]:
BS=8
ds = TensorDataset(X,masks,segments,ys)
dl = DataLoader(ds, batch_size=BS, shuffle=True)
for step, batch in enumerate(dl):
    X_b,masks_b,segments_b,ys_b = (o.to(device) for o in batch)
    print(X_b.shape, masks_b.shape,segments_b.shape,ys_b.shape)
    break

torch.Size([8, 512]) torch.Size([8, 512]) torch.Size([8, 512]) torch.Size([8, 4])


# model

In [451]:
BS=8
ds = TensorDataset(X,masks,segments,ys)
dl = DataLoader(ds, batch_size=BS, shuffle=True)
for step, batch in enumerate(dl):
    X_b,masks_b,segments_b,ys_b = (o.to(device) for o in batch)
    break

In [458]:
# https://huggingface.co/transformers/model_doc/bert.html#bertmodel
bertmodel=bertmodel.eval()
with torch.no_grad():
    last_hidden_state, pooler_output, hidden_states = bertmodel(X_b)
    avg_pool = torch.mean(last_hidden_state,1)
    max_pool,_ = torch.max(last_hidden_state,1)
last_hidden_state.shape,pooler_output.shape, len(hidden_states), avg_pool.shape, max_pool.shape

(torch.Size([8, 512, 768]),
 torch.Size([8, 768]),
 13,
 torch.Size([8, 768]),
 torch.Size([8, 768]))

In [ ]:
# see RobertaClassificationHead in transformers/modeling_roberta.py
N_HIDDEN = 768 
class TwtModel(nn.Module):
    def __init__(self, bertmodel, num_labels=4):
        super(TwtModel, self).__init__()
        self.bertmodel = bertmodel
        dense_size = 64
        self.relu=nn.ReLU(inplace=True)
        nx = N_HIDDEN+dense_size*3
        self.dense = nn.Linear(nx, nx)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(nx, num_labels)
        
    def forward(self, batch):
        X_b,masks_b,segments_b,ys_b = (o.to(device) for o in batch)
        attn_msk = (x!=tokenizer.pad_token_id).float()
        attn_msk = (x!=tokenizer.pad_token_id).float()

        
        hidden_features = self.mdl(x, attention_mask=attn_msk)[0]
        x = hidden_features[:, 0, :]  # take <s> token (equiv. to [CLS])        

        xcat = self.emb_cat(xcat)
        xcat = self.lin_cat(xcat)
        xcat = self.bn_cat(xcat)
        xcat = self.relu(xcat)

        xhst = self.emb_hst(xhst)
        xhst = self.lin_hst(xhst)
        xhst = self.bn_hst(xhst)
        xhst = self.relu(xhst)

        xfeat = self.lin_feat(xfeat)
        xfeat = self.bn_feat(xfeat)
        xfeat = self.relu(xfeat)
        
        x = torch.cat((x,xcat,xhst,xfeat),1)
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x